In [44]:
# Start using bag of words
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Stop spanish words
stop_words = ["de","la","que","el","en","y","a","los","del","se",
              "las","por","un","para","con","no","una","su","al","lo",
              "como","más","pero","sus","le","ya","o","este","sí","porque",
              "esta","entre","cuando","muy","sin","sobre","también","me",
              "hasta","hay","donde","quien","desde","todo","nos","durante",
              "todos","uno","les","ni","contra","otros","ese","eso","ante",
              "ellos","e","esto","mí","antes","algunos","qué","unos","yo","otro",
              "otras","otra","él","tanto","esa","estos","mucho","quienes","nada",
              "muchos","cual","poco","ella","estar","estas","algunas","algo",
              "nosotros","mi","mis","tú","te","ti","tu","tus","ellas","nosotras",
              "vosostros","vosostras","os","mío","mía","míos","mías","tuyo",
              "tuya","tuyos","tuyas","suyo","suya","suyos","suyas","nuestro",
              "nuestra","nuestros","nuestras","vuestro","vuestra","vuestros",
              "vuestras","esos","esas","estoy","estás","está","estamos","estáis",
              "están","esté","estés","estemos","estéis","estén","estaré","estarás",
              "estará","estaremos","estaréis","estarán","estaría","estarías",
              "estaríamos","estaríais","estarían","estaba","estabas","estábamos",
              "estabais","estaban","estuve","estuviste","estuvo","estuvimos",
              "estuvisteis","estuvieron","estuviera","estuvieras","estuviéramos",
              "estuvierais","estuvieran","estuviese","estuvieses","estuviésemos",
              "estuvieseis","estuviesen","estando","estado","estada","estados","estadas",
              "estad","he","has","ha","hemos","habéis","han","haya","hayas","hayamos",
              "hayáis","hayan","habré","habrás","habrá","habremos","habréis",
              "habrán","habría","habrías","habríamos","habríais","habrían","había",
              "habías","habíamos","habíais","habían","hube","hubiste","hubo","hubimos",
              "hubisteis","hubieron","hubiera","hubieras","hubiéramos","hubierais",
              "hubieran","hubiese","hubieses","hubiésemos","hubieseis","hubiesen",
              "habiendo","habido","habida","habidos","habidas","soy","eres","es",
              "somos","sois","son","sea","seas","seamos","seáis","sean","seré",
              "serás","será","seremos","seréis","serán","sería","serías","seríamos",
              "seríais","serían","era","eras","éramos","erais","eran","fui","fuiste",
              "fue","fuimos","fuisteis","fueron","fuera","fueras","fuéramos","fuerais","fueran",
              "fuese","fueses","fuésemos","fueseis","fuesen","sintiendo","sentido","sentida",
              "sentidos","sentidas","siente","sentid","tengo","tienes","tiene","tenemos",
              "tenéis","tienen","tenga","tengas","tengamos","tengáis","tengan","tendré",
              "tendrás","tendrá","tendremos","tendréis","tendrán","tendría","tendrías",
              "tendríamos","tendríais","tendrían","tenía","tenías","teníamos","teníais",
              "tenían","tuve","tuviste","tuvo","tuvimos","tuvisteis","tuvieron","tuviera",
              "tuvieras","tuviéramos","tuvierais","tuvieran","tuviese","tuvieses",
              "tuviésemos","tuvieseis","tuviesen","teniendo","tenido","tenida","tenidos",
              "tenidas","tened"]

# Read pandas csv
data = pd.read_csv('orders_orderitemspackage.csv', sep=',', names=None)
data.head()

,id,desc,price,weight,secure,created_at,box_id,order_id,code,qrcode,secure_amount
0,1,1 Certificados de nacimiento van dentro de un ...,1.0,1.0,1,2017-06-13 17:34:25,17,3291,IMD520CD7572954A3,/media/qr/_orders_package/orders-packageIMD520...,0.0
1,2,1 Certificados de nacimiento van dentro de un ...,1.0,1.0,0,2017-06-13 17:34:26,6,3291,IMC5645C4C56E341E,/media/qr/_orders_package/orders-packageIMC564...,0.0
2,3,1 Foto,1.0,1.0,0,2017-06-13 17:53:56,6,3297,IM551EEA624EB3434,/media/qr/_orders_package/orders-packageIM551E...,0.0
3,4,1 Cedula,1.0,1.0,0,2017-06-13 18:20:41,12,3311,IMEBC3EE076853485,/media/qr/_orders_package/orders-packageIMEBC3...,0.0
4,5,1 Cd,1.0,1.0,0,2017-06-13 19:44:39,21,3328,IMBE55852A744F4F6,/media/qr/_orders_package/orders-packageIMBE55...,0.0


In [119]:
# Split data in features and target
X = data['desc']
y = data['box_id']



In [166]:
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, learning_curve, StratifiedKFold, KFold, \
    validation_curve, cross_val_score

from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier



n = X.shape[0]
count = Counter(y.tolist())


def scorer(pred, classif, X_train, y_train): 
    #cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    cross_val=cross_val_score(
        classif, X_train, y_train, cv=10
    )
    

    print('CrossVal Acc:', np.mean(cross_val))
    print('Acc:', accuracy_score(y_test, pred))
    print('F1 Score:', f1_score(y_test, pred, average='micro'))

def fit_predict(classif, X_train, y_train):
    import time
    before = time.time()
    
    print('Model:', classif.__class__.__name__)   
    classif.fit(X_train, y_train)
    train_time =  time.time() - before
    
    y_pred = classif.predict(X_train)
    print('Train Acc: ',accuracy_score(y_train, y_pred))
    y_pred = classif.predict(X_test)
    print('Test Acc: ',accuracy_score(y_test, y_pred))
    pred_time = time.time() - before
    
    scorer(y_pred, classif, X_train, y_train)
    val_time= time.time() - before
    
    print('Train Time:', train_time)
    print('Pred Time:', pred_time)
    print('Validation Time:', val_time, '\n', '\n')

    return classif

def fit_grid_predict(classif, X_train, y_train, params, scoring='accuracy'):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    import time
    before = time.time()
    
    print('Model:', classif.__class__.__name__)
    gs = GridSearchCV(classif, params, scoring, cv=cv)
    gs.fit(X_train, y_train)
    train_time =  time.time() - before
    
    y_pred = gs.predict(X_train)
    y_pred = gs.predict(X_test)
    pred_time = time.time() - before
    
    print('Train Time:', train_time)
    print('Pred Time:', pred_time)
    print('Best Score:', gs.best_score_, '\n', '\n')

    return gs.best_estimator_


def complex_curve(estimator, X, y, **kwargs):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    # MODEL COMPLEX SCORE
    # Calculate the training and testing scores
    train_scores, test_scores = validation_curve(
        estimator, X, y, cv=cv, **kwargs
    )
    
     # Find the mean and standard deviation for smoothing
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    
    plt.figure(figsize=(7, 5))
    
    plt.title(estimator.__class__.__name__)
    plt.plot(kwargs['param_range'], train_mean, 'o-', color='r', label='Training Score')
    plt.plot(kwargs['param_range'], test_mean, 'o-', color='g', label='Validation Score')
    
    plt.legend(loc='lower right')
    plt.xlabel(kwargs['param_name'])
    plt.ylabel('score')
    plt.ylim([-0.05, 1.05])
    plt.show()
    
    # For each depth
    for x, k in enumerate(train_scores):
        print(kwargs['param_name'], kwargs['param_range'][x])
        print('score train:', np.mean(train_scores[x]))
        print('score test:', np.mean(test_scores[x]))
        
        
def learn_curve(estimator, X, y, **kwargs):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    # Generate the training set sizes increasing by 50
    train_sizes = np.rint(np.linspace(1, X.shape[0] * 0.8 - 1, 9)).astype(int)
        
    # Calculate the training and testing scores
    sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, train_sizes=train_sizes,
        scoring='accuracy'
    )

    # Find the mean and standard deviation for smoothing
    train_std = np.std(train_scores, axis=1)
    train_mean = np.mean(train_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    
    fig = plt.figure(figsize=(10, 7))
    # Subplot the learning curve
    ax = fig.add_subplot(2, 2,  1)
    ax.plot(sizes, train_mean, 'o-', color='r', label='Training Score')
    ax.plot(sizes, test_mean, 'o-', color='g', label='Testing Score')

    # Labels
    ax.set_title('%s = %s' % (kwargs['param_name'],0.1))
    ax.set_xlabel('Number of Training Points')
    ax.set_ylabel('Score')
    ax.set_xlim([0, X.shape[0] * 0.8])
    ax.set_ylim([-0.05, 1.05])
    
    # Visual aesthetics
    fig.show()
    plt.show()


In [167]:
# scaler = StandardScaler()
# scaler.fit(X_vector.toarray())
# X_vector = scaler.transform(X_vector.toarray())

In [168]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, Normalizer

from sklearn.pipeline import FeatureUnion, Pipeline

# vectorizer = CountVectorizer(
#     encoding='utf-8', 
#     strip_accents='unicode',
#     token_pattern=r'\b[^\d\W]+\b',
#     analyzer='word'
# )

def fit_vector(X_data):
    vectorizer = TfidfVectorizer(
        encoding='utf-8', 
        strip_accents='unicode',
        token_pattern=r'[a-zA-Z]{3,20}',
        analyzer='word',
        stop_words='english',
        use_idf=False, norm='l2'
    )

    #vectorizer = HashingVectorizer(n_features=20)
    return vectorizer.fit_transform(X_data), vectorizer

X_vector, vectorizer = fit_vector(X)

#print(vectorizer.stop_words_)
#print(vectorizer.vocabulary_)

{'certificados': 2991, 'nacimiento': 10302, 'van': 15681, 'dentro': 4604, 'mismo': 9918, 'sobre': 13998, 'foto': 6394, 'cedula': 2875, 'certificado': 2990, 'individuales': 7609, 'pita': 11825, 'bolso': 1851, 'negro': 10439, 'cuero': 4276, 'vestidos': 15848, 'nina': 10553, 'blanco': 1656, 'traje': 15226, 'folklor': 6335, 'para': 11254, 'camisa': 2518, 'falda': 5968, 'celeste': 2898, 'orillas': 10930, 'rojas': 13088, 'azul': 1117, 'amarillas': 464, 'verdes': 15809, 'color': 3567, 'anaranjado': 570, 'lila': 8713, 'gorras': 6864, 'negras': 10436, 'gris': 6959, 'camisola': 2545, 'blanca': 1648, 'letras': 8617, 'marca': 9296, 'jincho': 7945, 'camisetas': 2535, 'frente': 6475, 'traen': 15218, 'una': 15490, 'dice': 4796, 'nicaragua': 10514, 'escudo': 5622, 'bandera': 1272, 'nic': 10513, 'blancas': 1651, 'negra': 10432, 'tarjetas': 14715, 'invitacion': 7742, 'record': 12712, 'policia': 12024, 'bautismo': 1379, 'lentes': 8588, 'formularios': 6372, 'constancia': 3788, 'solicitud': 14062, 'credito

In [175]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelBinarizer

# binarizer = LabelBinarizer()
# binarizer.fit(y)
# y_ = binarizer.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_vector, y, test_size=0.2, random_state=0)

#estimator = SVC(kernel='linear', C=1)
#estimator = LogisticRegression(random_state=42, C=10, solver='sag', multi_class='multinomial')
# estimator = OneVsRestClassifier(LinearSVC())
# estimator.fit(X_train, y_train)
# y_pred = estimator.predict(X_test)

#linear_svc = fit_predict(OneVsRestClassifier(LinearSVC(C=0.01)), X_train, y_train)

#multi = fit_predict(MultinomialNB(), X_train, y_train)
#kn = fit_predict(KNeighborsClassifier(),X_train, y_train)
#sgd = fit_predict(SGDClassifier(eta0=0.00009, learning_rate='constant'), X_train, y_train)
#logit = fit_predict(LogisticRegression(random_state=42, solver='sag', multi_class='multinomial'), X_train, y_train)
d_tree = fit_predict(DecisionTreeClassifier(max_depth=4), X_train, y_train)
#r_f = fit_predict(RandomForestClassifier(), X_train, y_train)
#svc = fit_predict(SVC(kernel='linear', C=1), X_train, y_train)


Model: DecisionTreeClassifier
Train Acc:  0.654596826777
Test Acc:  0.656641168289
CrossVal Acc: 0.652860178463
Acc: 0.656641168289
F1 Score: 0.656641168289
Train Time: 0.29154515266418457
Pred Time: 0.30286574363708496
Validation Time: 2.687856674194336 
 



In [176]:
#learn_curve(linear_svc, X_vector, y, param_name='C')

In [177]:

# cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
# gs = GridSearchCV(LinearSVC(), [{
#     'C': [0.01,0.1,0.2,0.5,0.85,1.2,1.3,2.0,2.5,4.1,4.5,5.6,10.0],
#     'fit_intercept':[True, False], 
#     'dual':[True], 
#     'penalty':['l2'],
#     'loss':["hinge", "squared_hinge"]
# }, {
#     'C': [0.01,0.1,0.2,0.5,0.85,1.2,1.3,2.0,2.5,4.1,4.5,5.6,10.0],
#     'fit_intercept':[True, False], 
#     'dual':[False], 
#     'penalty':['l1'],
#     'loss':["squared_hinge"]
# }], cv=cv)


# gs.fit(X_train, y_train)


In [178]:
# print(gs.best_score_)
# print(gs.best_params_)
# print(gs.best_estimator_)

estimator = OneVsRestClassifier(LinearSVC(C=0.5, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))

fit_predict(OneVsRestClassifier(estimator), X_train, y_train)
#learn_curve(OneVsRestClassifier(estimator), X_train, y_train, param_name='C')

Model: OneVsRestClassifier
Train Acc:  0.763486198652
Test Acc:  0.689673157163
CrossVal Acc: 0.684505084449
Acc: 0.689673157163
F1 Score: 0.689673157163
Train Time: 0.9438674449920654
Pred Time: 0.9700548648834229
Validation Time: 9.436786651611328 
 



OneVsRestClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=0.5, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          n_jobs=1),
          n_jobs=1)

In [179]:
estimator.fit(X_train, y_train)
X_set = vectorizer.transform(['camisa zapatos '])
#v.vocabulary_
estimator.predict(X_set)

array([3])

In [101]:
#complex_curve(RandomForestClassifier(), X_vector, y, param_name='max_depth', param_range=[1,2,3,4,5])
#complex_curve(LogisticRegression(random_state=42, solver='sag', multi_class='multinomial'), X_vector, y, param_name='C', param_range=[1.0,1.1,1.2,1.3,1.4])